# Evaluating classification models

In this notebook, we will look at how to evaluate classification models in terms of the confusion matrix and measures such as accuracy, precision, and recall, as well as the ROC curve and AUC. 

We will, again, use the diabetes dataset that can be used to classify whether people have diabetes.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
diab_data = pd.read_csv('diabetes.csv')

In [ ]:
diab_data.head()

## Cross-validation

We saw last time that there was a very large variance for the K-Nearest-Neighbor classifier, in the sense that we got very different results dependent on the train-test split (based on what number we used for `random_state`). Let us first visualize that:

In [ ]:
y = diab_data["Outcome"]
X = diab_data.drop(columns = ["Age", "Outcome"])
acclist = [] 

random_seeds = np.random.randint(1000, size=50)
for seed in random_seeds:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)
    knn5 = KNeighborsClassifier(n_neighbors=5)
    knn5.fit(X_train, y_train)
    y_pred5_test = knn5.predict(X_test)
    acclist.append({"seed": seed, "Test accuracy": accuracy_score(y_test, y_pred5_test)})

accuracyDF = pd.DataFrame(acclist)
accuracyDF["Test accuracy"].describe()

In [ ]:
sns.histplot(data = accuracyDF, x = "Test accuracy")
plt.title("The distribution of test accuracy for 50 runs of KNN with K=5")
plt.plot()

Let us try to use cross-validation instead. There are multiple ways of doing this, but one is to use the `cross_val_score` from Scikit-learn model_selection module. It makes k-fold cross validation - that is, it trains k different models and evaluate the accuracy on the k hold-out folds. Note that, it only returns the accuracy scores of each fold. We should then train the model afterward on the entire set. (Strictly speaking, we should have done a train test split first and only ran the cross-validation on this training set. Then trained the model on the entire training set, and then finally evaluated the model on the untouched test set.)

In [ ]:
from sklearn.model_selection import cross_val_score

knn5 = KNeighborsClassifier(n_neighbors=5)
scores = cross_val_score(knn5, X, y, cv = 5)  # cv = 5 means 5-fold cross validation
scores

We can then use the mean of these as an unbiased estimate of the accuracy of our model on future unseen data:

In [ ]:
scores.mean()

Let us try out with different K's to see how much variance we have in this estimate:

In [ ]:
kacclist = [] 
for k in range(2, 20):
    scores = cross_val_score(knn5, X, y, cv = k)
    kacclist.append({"Folds": k, "Mean accuracy": scores.mean()})

kaccuracyDF = pd.DataFrame(kacclist)
kaccuracyDF["Mean accuracy"].describe()

In [ ]:
plt.plot(kaccuracyDF["Folds"], kaccuracyDF["Mean accuracy"], label = "Mean accurcay", color='blue', marker='o', linestyle='solid')    
plt.xlabel('Number of folds (k)')
plt.ylabel('Mean acuracy')
plt.title("Accuracy of different Ks")
plt.legend()

plt.show()

In [ ]:
sns.histplot(data = kaccuracyDF, x = "Mean accuracy")
plt.title("The distribution of mean accuracy for 2 to 20 k-fold cross-validation")
plt.plot()

We can now use 5-fold cross validation to chose a $K$ for the KNN classifier:

In [ ]:
def knnSweepCrossValidation(X, y, maxK, folds=5):
    kacclist = []
    
    for k in range(2, maxK):
        knn = KNeighborsClassifier(n_neighbors=k)     
        scores = cross_val_score(knn, X, y, cv = folds)
        kacclist.append({"K": k, "CV accuracy": scores.mean()})

    return pd.DataFrame(kacclist)

In [ ]:
knnSweepCrossValidation(X, y, 25)

Let us compare it to the `random_state=42` from last time.

In [ ]:
def knnSweep(X, y, maxK):
    accuracy_row_list = []
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    for k in range(2, maxK):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred_test = knn.predict(X_test)
        accuracy_test = accuracy_score(y_test, y_pred_test)
        accuracy_row_list.append({"K": k, "Random42 test accuracy": accuracy_test})

    return pd.DataFrame(accuracy_row_list)

In [ ]:
ran25 = knnSweep(X, y, 25)
cv25 = knnSweepCrossValidation(X, y, 25)

compareDF = pd.merge(ran25, cv25, on="K")
compareDF.head()

In [ ]:
plt.plot(compareDF["K"], compareDF["Random42 test accuracy"], label = 'Random 42 test accuracy', color='blue', marker='o', linestyle='solid')
plt.plot(compareDF["K"], compareDF["CV accuracy"], label = 'CV accurcay', color='red', marker='o', linestyle='solid')
    
plt.xlabel('K')
plt.ylabel('Accuracy')
plt.title("Accuracy of different Ks")
plt.legend()

plt.show()

We clearly see much less variance in the cross-validation accuracy! There is still variation, but that is mainly due to the fact that we have a small dataset and KNN is a model type with high variance. But it looks like that $K=10$ is a sensible choice.

## The confusion matrix

we will now focus on a single model of KNN trained for $K=5$ and $K=10$:

In [ ]:
knn5 = KNeighborsClassifier(n_neighbors=5)
knn5.fit(X, y)
y_pred5 = knn5.predict(X)

knn10 = KNeighborsClassifier(n_neighbors=10)
knn10.fit(X, y)
y_pred10 = knn10.predict(X)

We can use the `confusion_matrix` function from the Scikit-learn metrics submodule, to get a confusion matrix.

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y, y_pred5)

In [ ]:
confusion_matrix(y, y_pred10)

We can also make it visually more appealing with a heatmap style plot using `ConfusionMatrixDisplay` also from Scikit-learn metrics.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y, y_pred5)).plot()
plt.show()

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y, y_pred10)).plot()
plt.show()

It looks like the model for $K=10$ make less False Positive predictions (45 instead of 52) than the model for $K=5$. On the other hand, it makes much more False Negative predictions (136 instead of 98). So the $K=10$ model does not seem much better than the $K=5$ model. Let us calculate the various metrics to make a more detailed comparison.

If one wants to look at percentages instead, we can normalize the values in the cells to sum to one:

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y, y_pred10, normalize='all')).plot()
plt.show()

## Evaluation metrics

We can import all the revelant evaluation metrics from Scikit-learn metrics submodule:

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
EvaluationScoreDF = pd.DataFrame({"K": [5, 10],
                                  "Accuracy": [accuracy_score(y, y_pred5), accuracy_score(y, y_pred10)],
                                  "Precision": [precision_score(y, y_pred5), precision_score(y, y_pred10)],
                                  "Recall": [recall_score(y, y_pred5), recall_score(y, y_pred10)],
                                  "F1": [f1_score(y, y_pred5), f1_score(y, y_pred10)]})
EvaluationScoreDF                            

We see that the $K=5$ actually seems to perform better on all metrics.

## ROC and AUC evaluation

We will not look at the ROC curve and calculate AUC for the models. For this we will use our logistic regression model from last time as it naturally can give us class probabilities instead hard labels.

In [ ]:
from sklearn import linear_model

In [ ]:
logit_model = linear_model.LogisticRegression()

In [ ]:
X = diab_data[["Glucose", "BMI"]]  

In [ ]:
logit_model.fit(X, y)

We want to get the class probabilities (of class 1). We can use the method `.predict_proba` on the logistic model object:

In [ ]:
logit_model.predict_proba(X)

Note how it returns two columns of probabilities! The first column is the probability of class 0, while the second column is the probabilities of class 1 (note how the rows sum to 1). Thus, to get the probabilities for class 1, we can take the second column (denoted 1 in Python - I think I mixed it up last time):

In [ ]:
y_probs = logit_model.predict_proba(X)[:,1]
y_pred = logit_model.predict(X)

y_probs

To plot the ROC curve, we well use the `roc_curve` from Scikit-learn metrics to calculate the FPR and TPR for us that we can then plot

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
# Plotting the ROC Curve
fpr, tpr, thresholds = roc_curve(y, y_probs)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

We can also get the AUC directly using the `roc_auc_score` function from Scikit-learn:

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y, y_probs)